# O QUE O POTTER FAZ

* Pegao os peptídeos iniciais
* Realiza dockings
* Avalia todos as interações do peptídeo com a proteína usando VTR
* Contabiliza as interações
* Cria ranking de acordo com tipo de interações e percentual de ocupação
* Seleciona os melhores ranqueados
* Faz a troca dos aminoácidos que fazem contatos desfavoráveis
* Repete o processo

In [5]:
import os # Importa os, que permite passar comandos para o sistema
import sys
import re
import math
import time
import pymol
import shutil
import joblib
import pyrosetta
import numpy as np # Importa numpy, que é uma biblioteca voltada a cálculos matemáticos
import seaborn as sns  # Importa o seaborn, outra biblioteca para visualização de dados
import pandas as pd # Importa pandas, que é uma biblioteca para trabalhar com dados tabelados
import matplotlib.pyplot as plt # Importa o matplotlib.pyplot, que é uma biblioteca para visualização de dados
from multiprocessing import Process, Queue
from pymol import stored

In [6]:
pyrosetta.init("-seed_offset 1111")

PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python310.ubuntu 2023.03+release.35f325df8b7eab551e19ed8a4ace69a0c527c820 2023-01-18T16:04:22] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python310.ubuntu r338 2023.03+release.35f325df8b7 35f325df8b7eab551e19ed8a4ace69a0c527c820 http://www.pyrosetta.org 2023-01-18T16:04:22
core.init: command: PyRosetta -seed_offset 1111 -database /home/fcc073/miniconda3/lib/python3.10/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-143259817 seed_offset=1111 real_seed=-143258706
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-143258706 RG_type=mt19937


# Scripts auxiliares

## Funções para organizar e preprocessar entradas e visualizar resultados

In [7]:
# Este comando faz com que as tabelas sejam mostradas sem cortes
pd.set_option('display.max_rows', None)

# Essa função é para ler o arquivo mais rápido e com menos memória
def read_file(filename):
    for line in open(filename):
        yield line

# Esta função vai ordenar os textos corretamente como um humano ordenaria: 0, 1, 2, 3, 4, ..., 10, 11, 12, 13... e não 0, 1, 10, 11,12,...., 2, 3, 4, 5...
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

# Esta função serve para criar um lote de peptídeos para serem processados pelo Roseta em paralelo
def makeBatch(population, cores, add_fake = False):
    # População é a lista de strings representando as sequências dos peptídeos
    # cores é o número de processadores usados para fazer o docking.
    # O add_fake é porque o Rosetta algumas vezes ignora o primeiro peptídeo, por isso adicionei o BJOUXZ ao início 
    batches = list()
    x = int(len(population)/cores)
    if len(population) < cores:
        return [population]
    
    for i in range(x):
        if add_fake == True:
            new_batch = ["BJOUXZ"] # Este é um "peptídeo" falso para ser ignorado pelo Rosetta
        else:
            new_batch = list()
        for j in range(cores):
            pos = i*cores + j
            new_batch.append(population[pos])
        batches.append(new_batch)
        
    if len(population)%cores != 0: # If true, there is an excess of peptides not covered yet
        if add_fake == True:
            final_batch = ["BJOUXZ"]
        else:
            final_batch = list()
        
        for k in range(len(population)%cores):
            pos = x*cores + k
            final_batch.append(population[pos])
        batches.append(final_batch)
    return batches

# Esta função é utilizada para criar estruturas de peptídeos com torções e conformações aleatórias 
# Ela também cria o complexo peptídeo-alvo para realização do docking
def makePeptide(sequence, gen=0):
    if not os.path.isdir(f"Inputs/I{gen}/"):
        os.mkdir(f"Inputs/I{gen}/")
    peptide = pyrosetta.pose_from_sequence(sequence)
    angles = np.random.rand(3,len(sequence))*360 - 180
    for i in range(len(sequence)):
        peptide.set_phi(i+1, angles[0,i])
        peptide.set_psi(i+1, angles[1,i])
        #try:
        #    peptide.set_chi(1, i+1, angles[2,i])  
        #except:
        #    pass
    src = f"Temp/{sequence}.pdb"
    peptide.dump_pdb(src)
    dst = f"Inputs/I{gen}/{sequence}.pdb"
    pymol.cmd.load("template.pdb", "template") # Aqui carregamos a estrutura do template (neste caso a ACE2)
    pymol.cmd.load(src, "new")
    pymol.cmd.align("new", "template") # Aqui alinhamos o novo peptídeo criado com o template, para posicionar no sítio 
    pymol.cmd.save(src, selection = "new")
    pymol.cmd.reinitialize()
    pymol.cmd.load("target.pdb") # Aqui carregamos a estrutura do alvo em PDB
    pymol.cmd.load(src)
    pymol.cmd.alter('chain A', 'chain="B"') # É necessário trocar esses nomes, pois o Rosetta reconhece sempre cadeia A como
    pymol.cmd.alter('chain E', 'chain="A"') # receptora e cadeia B como peptídeo
    pymol.cmd.save(dst) # Salva arquivo do complexo alvo-peptídeo
    os.remove(src)
    pymol.cmd.reinitialize()
    return dst

# Esta função é para contar os contatos únicos que são realizados 
# O objetivo é calcular o percentual dos resíduos do sítio de ligação que estão "ocupados"
def countContacts(peptide_file, target = None, cutoff = 6.0, count_on = 'target'):
    # peptide_file é o arquivo PDB do complexo peptídeo-alvo depois do docking
    # Target é uma lista como a mostrada abaixo contendo os resíduos do sítio de ligação da proteína alvo
    # O parâmetro count_on pode ser ou target ou peptide e indica em qual dos dois serão contados os contatos
    if target == None:
        target = [('417', 'LYS'), ('446', 'GLY'), ('449', 'TYR'), ('453', 'TYR'), 
                  ('455', 'LEU'), ('456', 'PHE'), ('475', 'ALA'), ('476', 'GLY'),
                  ('477', 'SER'), ('486', 'PHE'), ('487', 'ASN'), ('489', 'TYR'),
                  ('490', 'PHE'), ('493', 'GLN'), ('494', 'SER'), ('495', 'TYR'),
                  ('496', 'GLY'), ('498', 'GLN'), ('500', 'THR'), ('501', 'ASN'),
                  ('502', 'GLY'), ('503', 'VAL'), ('505', 'TYR')] # Alterar para a RBM da Ana
    pymol.cmd.reinitialize()
    stored.list = []
    pymol.cmd.load(peptide_file)
    if count_on == 'target':
        pymol.cmd.indicate(''.join(['chain A within ', str(cutoff), ' of chain B']))
        pymol.cmd.iterate('indicate', 'stored.list.append((resi,resn))')
        pymol.cmd.delete('indicate')
        count = 0
        for aa in pd.unique(stored.list):
            if aa in target:
                count += 1
        occupancy = count/len(target)
        return occupancy
    elif count_on == 'peptide':
        pymol.cmd.indicate(''.join(['chain B within ', str(cutoff), ' of chain A']))
        pymol.cmd.iterate('indicate', 'stored.list.append((resi,resn))')
        pymol.cmd.delete('indicate')
        return pd.unique(stored.list)
    

# Esta é a função que realiza o docking em paralelo. Ela faz um docking por processador ao mesmo tempo.
def calculateFitness_Parallel(candidate, protocol, scorefxn, gen, reps, queue):
    best_pose = None
    best_fitness = 999999999
    scores = list() 
    for i in range(reps):
        # Carrega o complexo
        comp = pyrosetta.pose_from_pdb("Inputs/I" + str(gen) + "/comp(" + candidate + ").pdb")
        # Aplica o protocolo de docking e depois calcula o score
        protocol.apply(comp)
        fitness = scorefxn(comp)
        scores.append(fitness)
        if fitness < best_fitness:
            best_pose = comp.clone()
            best_fitness = fitness
    # Salva a melhor pose como PDB para ficar de registro
    best_pose.dump_pdb("Results/G" + str(gen) + "/dock(" + candidate + ").pdb")
    queue.put([candidate, scores])
    
    #Aqui só é para escrever no arquivo de registro
    scores_txt = [str(s) for s in scores]
    scores_txt = ",".join(scores_txt)
    p_info = open("Results/peptides_Potter.csv", mode="a")
    p_info.write("".join(["\n", str(gen), ",", candidate, scores_txt, ",", str(best_fitness)]))
    p_info.close()

# PepPlot é aquele plot que fizemos com as "colunas de peptídeos"
def PepPlot(data, ligcolumn, reccolumn, hue, palette, ax=None):
    receptors = list(natural_sort(pd.unique(data[reccolumn])))
    ligands = list(natural_sort(pd.unique(data[ligcolumn])))
    huetypes = list(pd.unique(data[hue]))
    figwidth = plt.gcf().get_figwidth()
    figheight = plt.gcf().get_figheight()
    dpi = plt.gcf().get_dpi()
    maxstack = data.groupby([reccolumn,ligcolumn]).count().groupby([reccolumn]).count().max()[0]
    maxwidth = figwidth*dpi/(3*len(receptors)*0.7)
    maxheight = figheight*dpi/maxstack
    if maxwidth < maxheight:
        maxsize = maxwidth
        top = maxwidth/maxheight
    else:
        top = 1
    print(top)
    maxsize = min([maxwidth,maxheight])
    ticks = np.linspace(0,1,len(receptors))
    yticks = np.linspace(0,top,maxstack)
    if ax==None:
        fig,ax = plt.subplots(figsize=(figwidth, figheight))
        ax.set_xlim(-0.2,1.2)
        ax.set_ylim(-0.2,1.2)
        ax.set_xticks(ticks,rotation=90)
        ax.set_xticklabels(receptors,rotation=90)
    else:
        ax.set_xticks(ticks)
        ax.set_xlim(-0.01,1.03)
        ax.set_ylim(-0.005,1.03)
        ax.set_xticklabels(receptors,rotation=90)
        i = 0
        for rec in receptors:
            j = 0
            used = []
            for lig in reversed(natural_sort(list(data.loc[data[reccolumn]==rec,ligcolumn]))):
                if lig not in used:
                    h = data.loc[(data[reccolumn]==rec)&(data[ligcolumn]==lig),hue].values[0]
                    k = huetypes.index(h)
                    c = palette[k]
                    ax.text(ticks[i]-0.01,yticks[j], lig.replace('_',''), fontsize=maxsize*0.9, color=c, fontname='monospace', fontweight='bold')
                    j += 1
                    used.append(lig)
            i += 1
            
        for i in range(len(huetypes)):
            ax.text(0.30*i+0.13, 1.05, huetypes[i],fontsize=maxsize*0.9, color=palette[i])

In [8]:
aminoletters = {'ALA':'A',
                'CYS':'C',
                'ASP':'D', 
                'GLU':'E', 
                'PHE':'F', 
                'GLY':'G',
                'HIS':'H',
                'ILE':'I',
                'LYS':'K',
                'LEU':'L',
                'MET':'M',
                'ASN':'N',
                'PRO':'P',
                'GLN':'Q',
                'ARG':'R',
                'SER':'S',
                'THR':'T',
                'VAL':'V',
                'TRP':'W',
                'TYR':'Y',
                }

conservative_list = {'A':['E', 'S', 'T'],
                     'C':['S', 'T','V'],
                     'D':['E'], 
                     'E':['A', 'D'], 
                     'F':['M', 'W', 'Y'], 
                     'G':['N', 'P'],
                     'H':['K', 'R'],
                     'I':['L', 'V'],
                     'K':['H', 'R'],
                     'L':['I', 'V'],
                     'M':['F', 'W', 'Y'],
                     'N':['G', 'D', 'Q'],
                     'P':['G'],
                     'Q':['N', 'E'],
                     'R':['H', 'K'],
                     'S':['A', 'T'],
                     'T':['A', 'I', 'S'],
                     'V':['A', 'I', 'L'],
                     'W':['F', 'M', 'Y'],
                     'Y':['F', 'M', 'W'],
                    }



# Precisaria completar para ser usável
nonconservative_list = {'A':['K'],
                     'C':[],
                     'D':[], 
                     'E':['K', 'Q'], 
                     'F':[], 
                     'G':[],
                     'H':['A'],
                     'I':[],
                     'K':['A', 'E'],
                     'L':[],
                     'M':[],
                     'N':['Y'],
                     'P':[],
                     'Q':[],
                     'R':[],
                     'S':[],
                     'T':[],
                     'V':[],
                     'W':[],
                     'Y':[],
                    }

RBM = []
for line in read_file('RBM.txt'):
    l = eval(line)
    RBM.append((l[0],aminoletters[l[1]]))
    
# Define the protocol
protocol = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
<SCOREFXNS>
    <ScoreFunction name="fa_standard" weights="ref2015.wts"/>
</SCOREFXNS>
<MOVERS>
    <FlexPepDock name="ppack"  ppk_only="true"/>
    <FlexPepDock name="fpd" lowres_abinitio="true" pep_refine="true"/>
    <FlexPepDock name="minimize"  min_only="true"/>
</MOVERS>
<PROTOCOLS>
    <Add mover="ppack"/>
    <Add mover="fpd"/>
    <Add mover="minimize"/>
</PROTOCOLS>
<OUTPUT/>
</ROSETTASCRIPTS>""").get_mover("ParsedProtocol")

# Define the scorefxn 
scorefxn = pyrosetta.create_score_function("ref2015") # This is the standard full_atom scoring from pyrosetta

protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: ...done
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsSchemaValidator: ...done
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="fa_standard" weights="ref2015.wts"/>
	</SCOREFXNS>
	<MOVERS>
		<FlexPepDock name="ppack" ppk_only="true"/>
		<FlexPepDock lowres_abinitio="true" name="fpd" pep_refine="true"/>
		<FlexPepDock min_only="true" name="minimize"/>
	</MOVERS>
	<PROTOCOLS>
		<Add mover="ppack"/>
		<Add mover="fpd"/>
		<Add mover="minimize"/>
	</PROTOCOLS>
	<OUTPUT/>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scorin

In [9]:
def random_variation(peptide, num_changes=1):
    n = len(peptide)
    pos = np.random.randint(n, size=num_changes)
    pep = list(peptide)
    for p in pos:
        res = peptide[p]
        pep[p] = np.random.choice(conservative_list[res])
    return ''.join(pep)  
    
def pre_check(nsims):
    if not os.path.isdir("Results/"):
        os.mkdir("Results/")
    if not os.path.isdir("Inputs/"):
        os.mkdir("Inputs/")
    if not os.path.isdir("Temp/"):
        os.mkdir("Temp/")
    if not os.path.isdir("Results/Docking/"):
        os.mkdir("Results/Docking/")
    if not os.path.isdir("Results/VTR/"):
        os.mkdir("Results/VTR/")
    if not os.path.isfile("Results/summary_Potter.csv"):
        summary = open("Results/summary_Potter.csv", mode="w+") # Create the summary csv file for later visualization of results
        summary.write("iteration,best peptide,score,%surface occupation,contacts,iteration time" + "\n") # The headers of summary
        summary.close()
    if not os.path.isfile("Results/peptides_Potter.csv"):
        p_info = open("Results/peptides_Potter.csv", mode="w+") # Create populations csv file for posterior checks
        p_info.write("iteration,sequence,") # The headers of summary
        for i in range(nsims):
            p_info.write(f"score_{i+1},")
        p_info.write("best,occupancy,contacts,hydrophobic,aromatic stacking,attractive,hydrogen bond,repulsive,salt bridge") 
        p_info.close()   


def start(pdb_file='6m0j.pdb', target_chain='E', ligand_chain='A', cutoff=12.0):
    pymol.cmd.reinitialize()
    # Checar se o arquivo que você indicou existe
    if os.path.isfile(pdb_file):
        pymol.cmd.load(pdb_file, 'base')
    else:
        pymol.cmd.fetch(pdb_file.split('.pdb')[0], 'base')
        
    # Seleciona todos os resíduos do receptor (neste caso ACE2) que estão a cutoff do nosso alvo (RBD)
    pymol.cmd.indicate(f'chain {ligand_chain} within {cutoff} of chain {target_chain}')
    peptide = pymol.cmd.get_fastastr(selection='indicate')
    pymol.cmd.save('template.pdb', selection ='indicate')
    pymol.cmd.indicate(f'chain {target_chain}')
    pymol.cmd.save('target.pdb', selection = 'indicate')
    pymol.cmd.reinitialize()
    return peptide.split('\n')[1]

def docking(protocol, scorefxn, sequence, gen=0, nsims=1):
    dst = f"Results/Docking/I{gen}/{sequence}.pdb"
    best_pose = None
    best_fitness = 999999999
    scores = list() 
    score_dict = {}
    for i in range(nsims):
        comp = pyrosetta.pose_from_pdb(f"Inputs/I0/{sequence}.pdb")
        # Aplica o protocolo de docking e depois calcula o score
        protocol.apply(comp)
        fitness = scorefxn(comp)
        scores.append(fitness)
        score_dict[f'{sequence}_{i}'] = fitness
        comp.dump_pdb(f'{dst[:-4]}_{i}.pdb')
        if fitness < best_fitness:
            best_pose = comp.clone()
            best_fitness = fitness
    # Salva a melhor pose como PDB para ficar de registro
    best_pose.dump_pdb(dst)
    scores_txt = [str(s) for s in scores]
    scores_txt = ",".join(scores_txt)
    p_info = open("Results/peptides_Potter.csv", mode="a")
    p_info.write("".join(["\n", str(gen), ",", sequence, scores_txt, ",", str(best_fitness)]))
    p_info.close()
    return score_dict

## TEST Scripts

In [10]:
def fake_docking(protocol=1, scorefxn=1, sequence=1, gen=0, nsims=1):
    origin = f"Inputs/I{gen}/{pept}.pdb"
    score_dict = {}
    for i in range(nsims):
        shutil.copy(src=origin, dst=f'Results/Docking/I{gen}/{pept}_{i}.pdb')
        fitness = np.random.choice([-1,1])*np.random.rand()*np.random.choice([50,100,150,200,250,300,350,400,450,500])
        score_dict[f'{sequence}_{i}'] = fitness
    return score_dict

def fake_VTR(entrada=0, outfile='fail.csv'):
    df = pd.read_csv('Results/ref.csv')
    df.to_csv(outfile, index=False)
    return None

def fake_countContacts(peptide_file=None, target = None, cutoff = 6.0, count_on = 'target'):
    return np.random.rand()

## Script do VTR, para processamento de resultados

In [11]:
'''
Script: contatos.py
Versão: 0.1
Função: Script simples para cálculos de contatos
Autor: @dcbmariano
Data: 2022
'''
# -----------------------------------------------------------------------------
# 0. DEFINIÇÕES
# -----------------------------------------------------------------------------

# INPUT: arquivo PDB de entrada
def VTR(entrada, outfile):
    mostrar_contato = {
                        'ligacao_hidrogenio': True,
                        'hidrofobico': True,
                        'aromatico': True,
                        'repulsivo': True,
                        'atrativo': True,
                        'ponte_salina': True
                        }

    # OUTPUT: saida = tela | csv
    saida = 'csv'

    # -----------------------------------------------------------------------------
    # Definições padrão do sistema
    # -----------------------------------------------------------------------------

    if saida == 'csv':
        w = open(outfile,'w')
        w.write('CONTACT;RES1:ATOM;RES2:ATOM;DIST\n') # cabeçalho do CSV

    # CONTATOS (baseado na definição do nAPOLI) 
    # tipo = (distancia_minima, distancia_maxima)
    aromatic = (2, 4)
    hidrogen_bond = (0, 3.9)
    hidrophobic = (2, 4.5)
    repulsive = (2, 6)
    atractive = (2, 6)
    salt_bridge = (0, 3.9)


    # REGRAS
    # 1 - deve ser feito por átomos de resíduos diferentes
    # 2 - aromatic = aromatic + aromatic
    # 3 - hb => aceptor + donor
    # 4 - hidrophobic: hidrofobic + hidrofobic
    # 5 - Repulsive: positive=>positive ou negative=>negative
    # 6 - Atractive: positive=>negative ou negative=>positive
    # 7 - salt_bridge: positive=>negative ou negative=>positive

    # 'RES:ATOM':[Hydrophobic,Aromatic,Positive,Negative,Donor,Acceptor]
    # 'ALA:CA':[0|1,0|1,0|1,0|1,0|1,0|1]
    contatos = { 'ALA:N': [0, 0, 0, 0, 1, 0], 'ALA:CA': [0, 0, 0, 0, 0, 0], 'ALA:C': [0, 0, 0, 0, 0, 0], 'ALA:O': [0, 0, 0, 0, 0, 1], 'ALA:CB': [1, 0, 0, 0, 0, 0], 'ARG:N': [0, 0, 0, 0, 1, 0], 'ARG:CA': [0, 0, 0, 0, 0, 0], 'ARG:C': [0, 0, 0, 0, 0, 0], 'ARG:O': [0, 0, 0, 0, 0, 1], 'ARG:CB': [1, 0, 0, 0, 0, 0], 'ARG:CG': [1, 0, 0, 0, 0, 0], 'ARG:CD': [0, 0, 0, 0, 0, 0], 'ARG:NE': [0, 0, 1, 0, 1, 0], 'ARG:CZ': [0, 0, 1, 0, 0, 0], 'ARG:NH1': [0, 0, 1, 0, 1, 0], 'ARG:NH2': [0, 0, 1, 0, 1, 0], 'ASN:N': [0, 0, 0, 0, 1, 0], 'ASN:CA': [0, 0, 0, 0, 0, 0], 'ASN:C': [0, 0, 0, 0, 0, 0], 'ASN:O': [0, 0, 0, 0, 0, 1], 'ASN:CB': [1, 0, 0, 0, 0, 0], 'ASN:CG': [0, 0, 0, 0, 0, 0], 'ASN:OD1': [0, 0, 0, 0, 0, 1], 'ASN:ND2': [0, 0, 0, 0, 1, 0], 'ASP:N': [0, 0, 0, 0, 1, 0], 'ASP:CA': [0, 0, 0, 0, 0, 0], 'ASP:C': [0, 0, 0, 0, 0, 0], 'ASP:O': [0, 0, 0, 0, 0, 1], 'ASP:CB': [1, 0, 0, 0, 0, 0], 'ASP:CG': [0, 0, 0, 0, 0, 0], 'ASP:OD1': [0, 0, 0, 1, 0, 1], 'ASP:OD2': [0, 0, 0, 1, 0, 1], 'CYS:N': [0, 0, 0, 0, 1, 0], 'CYS:CA': [0, 0, 0, 0, 0, 0], 'CYS:C': [0, 0, 0, 0, 0, 0], 'CYS:O': [0, 0, 0, 0, 0, 1], 'CYS:CB': [1, 0, 0, 0, 0, 0], 'CYS:SG': [0, 0, 0, 0, 1, 1], 'GLN:N': [0, 0, 0, 0, 1, 0], 'GLN:CA': [0, 0, 0, 0, 0, 0], 'GLN:C': [0, 0, 0, 0, 0, 0], 'GLN:O': [0, 0, 0, 0, 0, 1], 'GLN:CB': [1, 0, 0, 0, 0, 0], 'GLN:CG': [1, 0, 0, 0, 0, 0], 'GLN:CD': [0, 0, 0, 0, 0, 0], 'GLN:OE1': [0, 0, 0, 0, 0, 1], 'GLN:NE2': [0, 0, 0, 0, 1, 0], 'GLU:N': [0, 0, 0, 0, 1, 0], 'GLU:CA': [0, 0, 0, 0, 0, 0], 'GLU:C': [0, 0, 0, 0, 0, 0], 'GLU:O': [0, 0, 0, 0, 0, 1], 'GLU:CB': [1, 0, 0, 0, 0, 0], 'GLU:CG': [1, 0, 0, 0, 0, 0], 'GLU:CD': [0, 0, 0, 0, 0, 0], 'GLU:OE1': [0, 0, 0, 1, 0, 1], 'GLU:OE2': [0, 0, 0, 1, 0, 1], 'GLY:N': [0, 0, 0, 0, 1, 0], 'GLY:CA': [0, 0, 0, 0, 0, 0], 'GLY:C': [0, 0, 0, 0, 0, 0], 'GLY:O': [0, 0, 0, 0, 0, 1], 'HIS:N': [0, 0, 0, 0, 1, 0], 'HIS:CA': [0, 0, 0, 0, 0, 0], 'HIS:C': [0, 0, 0, 0, 0, 0], 'HIS:O': [0, 0, 0, 0, 0, 1], 'HIS:CB': [1, 0, 0, 0, 0, 0], 'HIS:CG': [0, 1, 0, 0, 0, 0], 'HIS:ND1': [0, 1, 1, 0, 1, 1], 'HIS:CD2': [0, 1, 0, 0, 0, 0], 'HIS:CE1': [0, 1, 0, 0, 0, 0], 'HIS:NE2': [0, 1, 1, 0, 1, 1], 'ILE:N': [0, 0, 0, 0, 1, 0], 'ILE:CA': [0, 0, 0, 0, 0, 0], 'ILE:C': [0, 0, 0, 0, 0, 0], 'ILE:O': [0, 0, 0, 0, 0, 1], 'ILE:CB': [1, 0, 0, 0, 0, 0], 'ILE:CG1': [1, 0, 0, 0, 0, 0], 'ILE:CG2': [1, 0, 0, 0, 0, 0], 'ILE:CD1': [1, 0, 0, 0, 0, 0], 'LEU:N': [0, 0, 0, 0, 1, 0], 'LEU:CA': [0, 0, 0, 0, 0, 0], 'LEU:C': [0, 0, 0, 0, 0, 0], 'LEU:O': [0, 0, 0, 0, 0, 1], 'LEU:CB': [1, 0, 0, 0, 0, 0], 'LEU:CG': [1, 0, 0, 0, 0, 0], 'LEU:CD1': [1, 0, 0, 0, 0, 0], 'LEU:CD2': [1, 0, 0, 0, 0, 0], 'LYS:N': [0, 0, 0, 0, 1, 0], 'LYS:CA': [0, 0, 0, 0, 0, 0], 'LYS:C': [0, 0, 0, 0, 0, 0], 'LYS:O': [0, 0, 0, 0, 0, 1], 'LYS:CB': [1, 0, 0, 0, 0, 0], 'LYS:CG': [1, 0, 0, 0, 0, 0], 'LYS:CD': [1, 0, 0, 0, 0, 0], 'LYS:CE': [0, 0, 0, 0, 0, 0], 'LYS:NZ': [0, 0, 1, 0, 1, 0], 'MET:N': [0, 0, 0, 0, 1, 0], 'MET:CA': [0, 0, 0, 0, 0, 0], 'MET:C': [0, 0, 0, 0, 0, 0], 'MET:O': [0, 0, 0, 0, 0, 1], 'MET:CB': [1, 0, 0, 0, 0, 0], 'MET:CG': [1, 0, 0, 0, 0, 0], 'MET:SD': [0, 0, 0, 0, 0, 1], 'MET:CE': [1, 0, 0, 0, 0, 0], 'PHE:N': [0, 0, 0, 0, 1, 0], 'PHE:CA': [0, 0, 0, 0, 0, 0], 'PHE:C': [0, 0, 0, 0, 0, 0], 'PHE:O': [0, 0, 0, 0, 0, 1], 'PHE:CB': [1, 0, 0, 0, 0, 0], 'PHE:CG': [1, 1, 0, 0, 0, 0], 'PHE:CD1': [1, 1, 0, 0, 0, 0], 'PHE:CD2': [1, 1, 0, 0, 0, 0], 'PHE:CE1': [1, 1, 0, 0, 0, 0], 'PHE:CE2': [1, 1, 0, 0, 0, 0], 'PHE:CZ': [1, 1, 0, 0, 0, 0], 'PRO:N': [0, 0, 0, 0, 0, 0], 'PRO:CA': [0, 0, 0, 0, 0, 0], 'PRO:C': [0, 0, 0, 0, 0, 0], 'PRO:O': [0, 0, 0, 0, 0, 1], 'PRO:CB': [1, 0, 0, 0, 0, 0], 'PRO:CG': [1, 0, 0, 0, 0, 0], 'PRO:CD': [0, 0, 0, 0, 0, 0], 'SER:N': [0, 0, 0, 0, 1, 0], 'SER:CA': [0, 0, 0, 0, 0, 0], 'SER:C': [0, 0, 0, 0, 0, 0], 'SER:O': [0, 0, 0, 0, 0, 1], 'SER:CB': [0, 0, 0, 0, 0, 0], 'SER:OG': [0, 0, 0, 0, 1, 1], 'THR:N': [0, 0, 0, 0, 1, 0], 'THR:CA': [0, 0, 0, 0, 0, 0], 'THR:C': [0, 0, 0, 0, 0, 0], 'THR:O': [0, 0, 0, 0, 0, 1], 'THR:CB': [0, 0, 0, 0, 0, 0], 'THR:OG1': [0, 0, 0, 0, 1, 1], 'THR:CG2': [1, 0, 0, 0, 0, 0], 'TRP:N': [0, 0, 0, 0, 1, 0], 'TRP:CA': [0, 0, 0, 0, 0, 0], 'TRP:C': [0, 0, 0, 0, 0, 0], 'TRP:O': [0, 0, 0, 0, 0, 1], 'TRP:CB': [1, 0, 0, 0, 0, 0], 'TRP:CG': [1, 1, 0, 0, 0, 0], 'TRP:CD1': [0, 1, 0, 0, 0, 0], 'TRP:CD2': [1, 1, 0, 0, 0, 0], 'TRP:NE1': [0, 1, 0, 0, 1, 0], 'TRP:CE2': [0, 1, 0, 0, 0, 0], 'TRP:CE3': [1, 1, 0, 0, 0, 0], 'TRP:CZ2': [1, 1, 0, 0, 0, 0], 'TRP:CZ3': [1, 1, 0, 0, 0, 0], 'TRP:CH2': [1, 1, 0, 0, 0, 0], 'TYR:N': [0, 0, 0, 0, 1, 0], 'TYR:CA': [0, 0, 0, 0, 0, 0], 'TYR:C': [0, 0, 0, 0, 0, 0], 'TYR:O': [0, 0, 0, 0, 0, 1], 'TYR:CB': [1, 0, 0, 0, 0, 0], 'TYR:CG': [1, 1, 0, 0, 0, 0], 'TYR:CD1': [1, 1, 0, 0, 0, 0], 'TYR:CD2': [1, 1, 0, 0, 0, 0], 'TYR:CE1': [1, 1, 0, 0, 0, 0], 'TYR:CE2': [1, 1, 0, 0, 0, 0], 'TYR:CZ': [0, 1, 0, 0, 0, 0], 'TYR:OH': [0, 0, 0, 0, 1, 1], 'VAL:N': [0, 0, 0, 0, 1, 0], 'VAL:CA': [0, 0, 0, 0, 0, 0], 'VAL:C': [0, 0, 0, 0, 0, 0], 'VAL:O': [0, 0, 0, 0, 0, 1], 'VAL:CB': [1, 0, 0, 0, 0, 0], 'VAL:CG1': [1, 0, 0, 0, 0, 0], 'VAL:CG2': [1, 0, 0, 0, 0, 0]}

    # -----------------------------------------------------------------------------
    # 1. LÊ ARQUIVO PDB E CONVERTE EM DICIONÁRIO
    # -----------------------------------------------------------------------------
    # lê átomos
    with open(entrada) as pdb_file:
        linhas = pdb_file.readlines()
        pdb = {}
        for linha in linhas:
            if linha[0:4] == 'ATOM':
                residuo = linha[17:20].strip()
                atomo = linha[13:16].strip()
                res_num = linha[22:26].strip()
                cadeia = linha[21]

                chave = cadeia+':'+res_num+'-'+residuo+':'+atomo

                x = float(linha[30:38].strip())
                y = float(linha[38:46].strip())
                z = float(linha[46:54].strip())

                coord = (x, y, z)

                pdb[chave] = coord

    # -----------------------------------------------------------------------------
    # 2. CALCULA matriz de distância => TODOS contra TODOS
    # -----------------------------------------------------------------------------

    def distancia(x1,y1,z1,x2,y2,z2):
        ''' Calcula a distância euclidiana usando a fórmula:
            dist² = (x2-x1)² + (y2-y1)² + (z2-z1)²
        '''
        dist = math.sqrt(
            (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2 
        )

        return round(dist, 1)

    matriz_distancia = {}

    # contatores numéricos
    ni = 0
    nj = 0


    # preenche linha
    for i in pdb:

        matriz_distancia[i] = {}

        # preenche coluna
        for j in pdb:

            if ni<nj:  # remove redundância (diagonal inferior)

                matriz_distancia[i][j] = distancia(
                    pdb[i][0], pdb[i][1], pdb[i][2],
                    pdb[j][0], pdb[j][1], pdb[j][2]
                )

            nj+=1
        ni+=1

    # -----------------------------------------------------------------------------
    # 3. CÁLCULO DE CONTATOS 
    # -----------------------------------------------------------------------------
    # contatos = {'RES:ATOM': [Hydrophobic, Aromatic, Positive, Negative, Donor, Acceptor]}

    # Analisa matriz de distância
    for i in matriz_distancia:

        r1 = i.split('-')
        r1_num = r1[0]
        r1_name = r1[1]

        for j in matriz_distancia[i]:

            r2 = j.split('-')
            r2_num = r2[0]
            r2_name = r2[1]

            # evita comparações com o mesmo resíduo
            if r1_num != r2_num:


                # hidrofobico -----------------------------------------------------
                if mostrar_contato['hidrofobico']:
                    if matriz_distancia[i][j] >= hidrophobic[0] and matriz_distancia[i][j] <= hidrophobic[1]:
                        try:
                            if contatos[r1_name][0] == 1 and contatos[r2_name][0] == 1:
                                if saida == 'tela':
                                    print('HIDROFÓBICO', i, j, matriz_distancia[i][j], sep=';')
                                else:
                                    w.write("HY;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


                # aromatic --------------------------------------------------------
                if mostrar_contato['aromatico']:
                    if matriz_distancia[i][j] >= aromatic[0] and matriz_distancia[i][j] <= aromatic[1]:
                        try:
                            if contatos[r1_name][1] == 1 and contatos[r2_name][1] == 1:
                                if saida == 'tela':
                                    print('AROMÁTICO', i, j, matriz_distancia[i][j], sep=';')
                                else:
                                    w.write("AR;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


                # repulsive -------------------------------------------------------
                if mostrar_contato['repulsivo']:
                    if matriz_distancia[i][j] >= repulsive[0] and matriz_distancia[i][j] <= repulsive[1]:
                        try:
                            if ( 
                                (contatos[r1_name][2] == 1 and contatos[r2_name][2] == 1) or # positivos vs. positivo
                                (contatos[r1_name][3] == 1 and contatos[r2_name][3] == 1)    # negativo vs. negativo
                            ):
                                if saida == 'tela':
                                    print('REPULSIVO', i, j, matriz_distancia[i][j], sep=';')
                                else:
                                    w.write("RE;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


                # atrativo -------------------------------------------------------
                if mostrar_contato['atrativo']:
                    if matriz_distancia[i][j] >= repulsive[0] and matriz_distancia[i][j] <= repulsive[1]:
                        try:
                            if ( 
                                (contatos[r1_name][2] == 1 and contatos[r2_name][3] == 1) or # positivos vs. negativo
                                (contatos[r1_name][3] == 1 and contatos[r2_name][2] == 1)    # negativo vs. positivo
                            ):
                                if saida == 'tela':
                                    print('ATRATIVO', i, j, matriz_distancia[i][j], sep=';')
                                else:
                                    w.write("AT;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


                # ligação de hidrogênio (hb) ------------------------------------
                if mostrar_contato['ligacao_hidrogenio']:
                    if matriz_distancia[i][j] >= hidrogen_bond[0] and matriz_distancia[i][j] <= hidrogen_bond[1]:
                        try:
                            if ( 
                                (contatos[r1_name][4] == 1 and contatos[r2_name][5] == 1) or # donor vs. aceptor
                                (contatos[r1_name][5] == 1 and contatos[r2_name][4] == 1)    # aceptor vs. donor
                            ):
                                if saida == 'tela':
                                    print('LIGAÇÃO_DE_HIDROGÊNIO', i, j, matriz_distancia[i][j], sep=';')
                                else:
                                    w.write("HB;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


                # ponte salina (atrativo + hb) ------------------------------------
                if mostrar_contato['ponte_salina']:
                    if matriz_distancia[i][j] >= salt_bridge[0] and matriz_distancia[i][j] <= salt_bridge[1]:
                        try:
                            # atrativo
                            if ( 
                                (contatos[r1_name][2] == 1 and contatos[r2_name][3] == 1) or # positivos vs. negativo
                                (contatos[r1_name][3] == 1 and contatos[r2_name][2] == 1)    # negativo vs. positivo
                            ):
                                # ligação de hidrogênio
                                if ( 
                                    (contatos[r1_name][4] == 1 and contatos[r2_name][5] == 1) or # donor vs. aceptor
                                    (contatos[r1_name][5] == 1 and contatos[r2_name][4] == 1)    # aceptor vs. donor
                                ):
                                    if saida == 'tela':
                                        print('PONTE_SALINA', i, j, matriz_distancia[i][j], sep=';')
                                    else:
                                        w.write("SB;"+i+";"+j+";"+str(matriz_distancia[i][j])+"\n")
                        except:
                            warning = 'Ignorando chave inexistente: '+i+','+j


    if(saida == 'csv'):
        print('Cálculo de contatos realizado com sucesso. \nResultado gravado em: "contato.csv"')
        print('\nEspecificações do CSV:\nTipo de contato; Resíduo:átomo 1; Resíduo:átomo 2; Distância\n')
        print('Tipos de contatos:\n\tHB: ligação de hidrogênio\n\tHY: hidrofóbio\n\tAR: aromático\n\tAT: atrativo\n\tRE: repulsivo\n\tSB: ponte salida\n')
        

def processa_VTR(gen):
    # É preciso identificar qual peptídeo que originou o contato
    # Criamos dicionários para qualquer as informações que nos importam do VTR
    df = {'Protein':[],
          'Peptide':[],
          'Distance':[],
          'Contact':[],
          'File': []
         }

    modelos = {'Iteration':[],
               'Peptide':[],
               'Hydrophobic':[],
               'Aromatic Stacking':[],
               'Attractive':[],
               'Hydrogen Bond':[],
               'Repulsive':[],
               'Salt Bridge':[],
    }

    # Criamos um dicionário para padronizar os tipos de ligação
    new = {"HB": 'Hydrogen Bond',
           "HY": 'Hydrophobic',
           "AR": 'Aromatic Stacking',
           "AT": 'Attractive',
           "RE": 'Repulsive',
           "SB": 'Salt Bridge',
          }

    # Indicamos o nome da pasta onde os resultados do VTR a serem tratados estão
    pasta = f'Results/VTR/I{gen}/'

    # O Loop/laço for a seguir lê cada linha dos arquivos do VTR e as interpreta
    for file in os.listdir(pasta):
        filename = pasta + file
        dist = False
        pep_name = file.split('.')[0]
        modelos['Peptide'].append(pep_name) #Adicionamos o nome do peptídeo
        modelos['Iteration'].append(gen)
        # Inicializamos as contagens dos tipos de interação que o VTR identificou
        tipos = {'Hydrophobic':0,
                 'Aromatic Stacking':0,
                 'Attractive':0,
                 'Hydrogen Bond':0,
                 'Repulsive':0,
                 'Salt Bridge':0,
                }

        # Neste laço/loop for, nós preenchemos nossa tabela com as informações interpretadas
        for line in read_file(filename):
            l = re.split(';|:|-|\n',line) # Dividimos a linha em palavras, ignorando os espaços
            if len(l) == 11 and l[1] in ['A','B']: # O VTR tem o arquivo padronizado. Linhas com informações que queremos tem 11 campos
                if l[1] != l[5]: # Se a cadeia 1 l[2] é diferente da cadeia 2 l[7], processamos
                    if l[1] == 'A':
                        df['Protein'].append(' '.join(l[2:4]))
                        df['Peptide'].append(' '.join(l[6:8]))
                    else:
                        df['Protein'].append(' '.join(l[6:8]))
                        df['Peptide'].append(' '.join(l[2:4]))
                    df['Distance'].append(l[9])
                    df['Contact'].append(new[l[0]])
                    df['File'].append(file)
                    tipos[new[l[0]]] += 1  # Contabilizamos o tipo de contato encontrado

        # Neste laço/loop for, nós registramos as contagens de tipos de ligação para o modelo
        for k in tipos.keys():
            modelos[k].append(tipos[k])

    # CRIAR A TABELA
    df = pd.DataFrame(df)
    modelos = pd.DataFrame(modelos)
    return df, modelos

# Script principal - Geração de peptídeos e docking

In [21]:
#Criar pastas, caso não existam
nsims = 3 #Número de dockings por peptídeo (recomendável 50)
pop_size = 10 #Número de peptídeos a serem gerados
iterations = 10 #Número de iterações que deverão ser feitas
gen = 0 #Não modificar!
pre_check(nsims)
if not os.path.isdir(f"Results/Docking/I{gen}/"):
    os.mkdir(f"Results/Docking/I{gen}/")
if not os.path.isdir(f"Results/VTR/I{gen}/"):
    os.mkdir(f"Results/VTR/I{gen}/")

testados = []
#Passo 1 - Cortar peptídeo base
#peptide = start('6m0j.pdb')
peptide = 'STIEEQAKTFLDKFNHEAEDLFYQSSLASWN'
testados.append(peptide)

#Passo 2 - Docking
makePeptide(peptide, gen=0)
scores_dict = docking(protocol, scorefxn, peptide, gen=0, nsims=nsims)

#Passo 3 - Contar quantas interações são realizadas
site = '['
for s in open('RBM.txt').readlines():
    site = site + s.replace('\n',',')
site = site + ']'
site = eval(site)
ocup_dict = {}
modelos = pd.DataFrame()
for i in range(nsims):
    peptide_file = f"Results/Docking/I{gen}/{peptide}_{i}.pdb"
    occupancy = countContacts(peptide_file, target=site, cutoff=6.0)
    ocup_dict[f'{peptide}_{i}'] = occupancy
    #Passo 4 - Identificar os tipos de interações com o VTR
    VTR(entrada=peptide_file, outfile=f'Results/VTR/I{gen}/{peptide}_{i}.csv')

df, modelos = processa_VTR(gen)
modelos['Contatos'] = 0
modelos['Ocup'] = 0
modelos['Scores'] = 0
df['iteration'] = gen
for i in range(nsims):
    contatos = modelos.loc[modelos['Peptide']==f'{peptide}_{i}', ['Hydrophobic', 'Aromatic Stacking', 'Attractive', 'Hydrogen Bond', 'Repulsive', 'Salt Bridge']].to_numpy().sum() 
    modelos.loc[modelos['Peptide']==f'{peptide}_{i}', 'Contatos'] = contatos
    modelos.loc[modelos['Peptide']==f'{peptide}_{i}', 'Ocup'] = ocup_dict[f'{peptide}_{i}']
    modelos.loc[modelos['Peptide']==f'{peptide}_{i}', 'Scores'] = scores_dict[f'{peptide}_{i}']

df.to_csv(f'Results/contacts_{gen}.csv')
modelos.to_csv('Results/Modelos.csv', index=False)

#Passo 5 - Identifica porção que interage com a RBM
# O critério usado aqui é a máxima ocupação
max_ocup_ite = modelos.loc[modelos['Iteration']==gen, 'Ocup'].max()
pep = modelos.loc[modelos['Ocup']==max_ocup_ite, 'Peptide'].values[0]
pep_file  = f"Results/Docking/I{gen}/{pep}.pdb"
residuos = countContacts(pep_file, target = RBM, cutoff = 6.0, count_on = 'peptide')

#Passo 6 - Identificar região que interage e clivar as partes que não tem interação para construção de novos peptídeos
em_contato = ''.join([aminoletters[y] for x,y in residuos])

population = []
# Precisa definir como será feito no caso de não ser aleatório - A escolha de quantas modificações 
# e de quais resíduos precisarão ser modificados. E como
while len(population) < pop_size:
    num_changes = np.random.randint(low=1, high=len(em_contato))
    new_pep = random_variation(em_contato, num_changes)
    if new_pep not in testados:
        population.append(new_pep)
        testados.append(new_pep)
"""
# Início das iterações:
for gen in range(1, iterations):
    if not os.path.isdir(f"Results/Docking/I{gen}/"):
        os.mkdir(f"Results/Docking/I{gen}/")
    if not os.path.isdir(f"Results/VTR/I{gen}/"):
        os.mkdir(f"Results/VTR/I{gen}/")
    
    
    for pept in population:
        makePeptide(pept, gen=gen)
        scores = docking(protocol, scorefxn, pept, gen=gen, nsims=nsims)
        scores_dict |= scores
        for i in range(nsims):
            peptide_file = f"Results/Docking/I{gen}/{pept}_{i}.pdb"
            VTR(entrada=peptide_file, outfile=f'Results/VTR/I{gen}/{pept}_{i}.csv')
    
    
    for peptide_file in os.listdir(f"Results/Docking/I{gen}/"):
        for i in range(nsims):
            peptide_file = f"Results/Docking/I{gen}/{peptide_file}"
            #residuos = countContacts(peptide_file, target = RBM, cutoff = 6.0, count_on = 'peptide')
    
            #Passo 4 - Contar quantas interações são realizadas
            occupancy = countContacts(peptide_file, target=site, cutoff=6.0)
            pep = peptide_file.split('/')[-1].split('.')[0]
            ocup_dict[pep] = occupancy
        
    #Passo 5 - Identificar os tipos de interações com o VTR
    old_model = modelos.copy()
    df, modelos = processa_VTR(gen)
    modelos['Contatos'] = 0
    modelos['Ocup'] = 0
    modelos['Scores'] = 0
    for pept in population:
        for i in range(nsims):
            contatos = modelos.loc[modelos['Peptide']==f'{pept}_{i}', ['Hydrophobic', 'Aromatic Stacking', 'Attractive', 'Hydrogen Bond', 'Repulsive', 'Salt Bridge']].to_numpy().sum() 
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Contatos'] = contatos
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Ocup'] = ocup_dict[f'{pept}_{i}']
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Scores'] = scores_dict[f'{pept}_{i}']
        
    df['iteration'] = gen
    modelos = pd.concat([old_model, modelos]).reset_index(drop=True)
    modelos.to_csv('Results/Modelos.csv', index=False)
    df.to_csv(f'Results/contacts_{gen}.csv')
    
    # Gera nova população baseada no peptídeo que teve maior ocupação
    max_ocup_ite = modelos.loc[modelos['Iteration']==gen, 'Ocup'].max()
    pep = modelos.loc[modelos['Ocup']==max_ocup_ite, 'Peptide'].values[0]
    melhor = pep.split('_')[0]

    population = []
    # Precisa definir como será feito no caso de não ser aleatório - A escolha de quantas modificações 
    # e de quais resíduos precisarão ser modificados. E como
    while len(population) < pop_size:
        num_changes = np.random.randint(low=1, high=len(melhor))
        new_pep = random_variation(melhor, num_changes)
        if new_pep not in testados:
            population.append(new_pep)
            testados.append(new_pep)
    
"""
#Passo 7 - Selecionar os melhores peptídeos segundo critérios:
    # a) Afinidade do docking
    # b) Nº de contatos (átomo x átomo) favoráveis (Atrativo, empilhamento aromático, ponte de sulfeto, ligação de hidrogênio)
    # c) Ocupação da RBM (% de resíduos)
    # d) Tipo de ligação
    #Atrativo, empilhamento aromático, ponte de sulfeto, ligação de hidrogênio, hidrofóbica, repulsivas
    #Contagem de ligações não é contagem de resíduos que interagem. Cada interação entre átomos é uma interação.    

#Passo 8 - Definir quantas modificações serão realizadas de acordo com as tabela de substituições
            # De acordo com a tabela de substituições
            # Fazer um algoritmo que realize todas as modificações segundo a tabela
            # Duas opções: Substituir todos os resíduos de uma vez ou Substituir somente um dos resíduos 
            # Digamos que temos 5 alaninas que podem ser substituidas. Substituímos todas? Ou somente as que não interagem? 
    
#Passo 9 - Realiza as modificações segundo tabela
            # CRIAR TODAS AS COMBINAÇÕES POSSÍVEIS (Calcular quantas possibilidades tem)
            # Gerar peptídeos com as substituições conservadoras e as não-conservadoras
    
#Passo 10 - Fazer docking dos novos peptídeos

#Passo 11 - Repetir os passos 3 a 10 por n iterações

#Passo 12 - Avaliar resultados

core.import_pose.import_pose: File 'Inputs/I0/STIEEQAKTFLDKFNHEAEDLFYQSSLASWN.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'NAG' from the PDB components dictionary for residue type 'pdb_NAG'
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 194
core.conformation.Conformation: [ WARNING ] missing heavyatom: O1 on residue pdb_NAG 195
core.conformation.Conformation: Found disulfide between residues 4 29
core.conformation.Conformation: current variant for 4 CYS
core.conformation.Conformation: current variant for 29 CYS
core.conformation.Conformation: current variant for 4 CYD
core.conformation.Conformation: current variant for 29 CYD
core.conformation.Conformation: Found disulfide between residues 47 100
core.conformation.Conformation: current variant for 47 CYS
core.conformation.Conformation: current variant for 100 CYS
core.conformation.Conformation: current 

FlexPepDockingProtocol:  residue 490 A  is at the interface
FlexPepDockingProtocol:  residue 491 A  is at the interface
FlexPepDockingProtocol:  residue 493 A  is at the interface
FlexPepDockingProtocol:  residue 494 A  is at the interface
FlexPepDockingProtocol:  residue 496 A  is at the interface
FlexPepDockingProtocol:  residue 498 A  is at the interface
FlexPepDockingProtocol:  residue 500 A  is at the interface
FlexPepDockingProtocol:  residue 501 A  is at the interface
FlexPepDockingProtocol:  residue 1 B  is at the interface
FlexPepDockingProtocol:  residue 11 B  is at the interface
FlexPepDockingProtocol:  residue 12 B  is at the interface
FlexPepDockingProtocol:  residue 13 B  is at the interface
FlexPepDockingProtocol:  residue 14 B  is at the interface
FlexPepDockingProtocol:  residue 15 B  is at the interface
FlexPepDockingProtocol:  residue 16 B  is at the interface
FlexPepDockingProtocol:  residue 17 B  is at the interface
FlexPepDockingProtocol:  residue 19 B  is at the 

core.pack.pack_rotamers: built 541 rotamers at 64 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 603 rotamers at 54 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: ini

core.pack.pack_rotamers: built 555 rotamers at 49 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 588 rotamers at 54 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: ini

FlexPepDockingPoseMetrics: 197 bsa: 0.60815 HB: 0 pack: 0.584257 unsat: 0
FlexPepDockingPoseMetrics: 198 bsa: 0 HB: 0 pack: 0.419841 unsat: 0
FlexPepDockingPoseMetrics: 199 bsa: 0 HB: 0 pack: 0.465055 unsat: 0
FlexPepDockingPoseMetrics: 200 bsa: 0 HB: 0 pack: 0.690516 unsat: 0
FlexPepDockingPoseMetrics: 201 bsa: 0 HB: 0 pack: 0.516966 unsat: 0
FlexPepDockingPoseMetrics: 202 bsa: 0 HB: 0 pack: 0.239656 unsat: 0
FlexPepDockingPoseMetrics: 203 bsa: 124.633 HB: 0 pack: 0.172436 unsat: 1
FlexPepDockingPoseMetrics: 204 bsa: 30.2014 HB: 0 pack: 0.606959 unsat: 1
FlexPepDockingPoseMetrics: 205 bsa: 49.4179 HB: 0 pack: 0.701244 unsat: 0
FlexPepDockingPoseMetrics: 206 bsa: 1.34041 HB: 0 pack: 0.136708 unsat: 0
FlexPepDockingPoseMetrics: 207 bsa: 0 HB: 0 pack: 0.0734377 unsat: 0
FlexPepDockingPoseMetrics: 208 bsa: 0 HB: 0 pack: 0.596797 unsat: 0
FlexPepDockingPoseMetrics: 209 bsa: 52.5208 HB: 0 pack: 0.758938 unsat: 1
FlexPepDockingPoseMetrics: 210 bsa: 73.864 HB: 0 pack: 0.362121 unsat: 0
FlexPe

...

FlexPepDockingPoseMetrics: 205 bsa: 24.6671 HB: 0 pack: 0.876865 unsat: 0
FlexPepDockingPoseMetrics: 206 bsa: 0 HB: 0 pack: 0.252648 unsat: 0
FlexPepDockingPoseMetrics: 207 bsa: 60.1729 HB: 0 pack: 0.144872 unsat: 1
FlexPepDockingPoseMetrics: 208 bsa: 125.127 HB: 0 pack: 0.939577 unsat: 0
FlexPepDockingPoseMetrics: 209 bsa: 72.2007 HB: 0 pack: 0.690903 unsat: 1
FlexPepDockingPoseMetrics: 210 bsa: 114.433 HB: 0 pack: 0.834878 unsat: 2
FlexPepDockingPoseMetrics: 211 bsa: 13.8218 HB: 0 pack: 0.417397 unsat: 1
FlexPepDockingPoseMetrics: 212 bsa: 126.452 HB: 0 pack: 0.742348 unsat: 0
FlexPepDockingPoseMetrics: 213 bsa: 0 HB: 0 pack: 0.509154 unsat: 0
FlexPepDockingPoseMetrics: 214 bsa: 23.3046 HB: 0 pack: 0.240683 unsat: 0
FlexPepDockingPoseMetrics: 215 bsa: 114.778 HB: 1 pack: 0.909435 unsat: 1
FlexPepDockingPoseMetrics: 216 bsa: 51.2531 HB: 0 pack: 0.140647 unsat: 0
FlexPepDockingPoseMetrics: 217 bsa: 13.4945 HB: 0 pack: 0.670808 unsat: 2
FlexPepDockingPoseMetrics: 218 bsa: 99.9337 HB: 0 

core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 485 rotamers at 47 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 452 rotamers at 55 positions.
core.

core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 470 rotamers at 41 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 468 rotamers at 40 positions.
core.

FlexPepDockingPoseMetrics: 198 bsa: 35.609 HB: 0 pack: 0.391041 unsat: 0
FlexPepDockingPoseMetrics: 199 bsa: 0 HB: 0 pack: 0.355799 unsat: 0
FlexPepDockingPoseMetrics: 200 bsa: 0 HB: 0 pack: 0.473423 unsat: 0
FlexPepDockingPoseMetrics: 201 bsa: 20.2939 HB: 0 pack: 0.261306 unsat: 0
FlexPepDockingPoseMetrics: 202 bsa: 48.3559 HB: 0 pack: 0.248806 unsat: 1
FlexPepDockingPoseMetrics: 203 bsa: 6.19495 HB: 1 pack: 0.301703 unsat: 0
FlexPepDockingPoseMetrics: 204 bsa: 71.4746 HB: 1 pack: 0.736701 unsat: 0
FlexPepDockingPoseMetrics: 205 bsa: 72.4149 HB: 0 pack: 0.908937 unsat: 0
FlexPepDockingPoseMetrics: 206 bsa: 82.4333 HB: 0 pack: 0.690523 unsat: 1
FlexPepDockingPoseMetrics: 207 bsa: 0 HB: 0 pack: 0.846804 unsat: 0
FlexPepDockingPoseMetrics: 208 bsa: 5.12343 HB: 0 pack: 0.18775 unsat: 0
FlexPepDockingPoseMetrics: 209 bsa: 132.222 HB: 0 pack: 0.499626 unsat: 1
FlexPepDockingPoseMetrics: 210 bsa: 1.49129 HB: 0 pack: 0.35698 unsat: 0
FlexPepDockingPoseMetrics: 211 bsa: 0 HB: 0 pack: 0.469517 

'\n# Início das iterações:\nfor gen in range(1, iterations):\n    if not os.path.isdir(f"Results/Docking/I{gen}/"):\n        os.mkdir(f"Results/Docking/I{gen}/")\n    if not os.path.isdir(f"Results/VTR/I{gen}/"):\n        os.mkdir(f"Results/VTR/I{gen}/")\n    \n    \n    for pept in population:\n        makePeptide(pept, gen=gen)\n        scores = docking(protocol, scorefxn, pept, gen=gen, nsims=nsims)\n        scores_dict |= scores\n        for i in range(nsims):\n            peptide_file = f"Results/Docking/I{gen}/{pept}_{i}.pdb"\n            VTR(entrada=peptide_file, outfile=f\'Results/VTR/I{gen}/{pept}_{i}.csv\')\n    \n    \n    for peptide_file in os.listdir(f"Results/Docking/I{gen}/"):\n        for i in range(nsims):\n            peptide_file = f"Results/Docking/I{gen}/{peptide_file}"\n            #residuos = countContacts(peptide_file, target = RBM, cutoff = 6.0, count_on = \'peptide\')\n    \n            #Passo 4 - Contar quantas interações são realizadas\n            occupanc

In [53]:
nsims = 3
iterations = 5
# Início das iterações:
for gen in range(1, iterations):
    if not os.path.isdir(f"Results/Docking/I{gen}/"):
        os.mkdir(f"Results/Docking/I{gen}/")
    if not os.path.isdir(f"Results/VTR/I{gen}/"):
        os.mkdir(f"Results/VTR/I{gen}/")
    
    for pept in population:
        makePeptide(pept, gen=gen)
        scores = fake_docking(protocol, scorefxn, pept, gen=gen, nsims=nsims)
        scores_dict |= scores
        for i in range(nsims):
            peptide_file = f"Results/Docking/I{gen}/{pept}_{i}.pdb"
            fake_VTR(entrada=peptide_file, outfile=f'Results/VTR/I{gen}/{pept}_{i}.csv')
    
    for peptide_file in os.listdir(f"Results/Docking/I{gen}/"):
        for i in range(nsims):
            peptide_file = f"Results/Docking/I{gen}/{peptide_file}"
            #residuos = countContacts(peptide_file, target = RBM, cutoff = 6.0, count_on = 'peptide')
    
            #Passo 4 - Contar quantas interações são realizadas
            occupancy = fake_countContacts(peptide_file, target=site, cutoff=6.0)
            pep = peptide_file.split('/')[-1].split('.')[0]
            ocup_dict[pep] = occupancy
        
    #Passo 5 - Identificar os tipos de interações com o VTR
    old_model = modelos.copy()
    df, modelos = processa_VTR(gen)
    modelos['Contatos'] = 0
    modelos['Ocup'] = 0
    modelos['Scores'] = 0
    for pept in population:
        for i in range(nsims):
            contatos = modelos.loc[modelos['Peptide']==f'{pept}_{i}', ['Hydrophobic', 'Aromatic Stacking', 'Attractive', 'Hydrogen Bond', 'Repulsive', 'Salt Bridge']].to_numpy().sum() 
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Contatos'] = contatos
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Ocup'] = ocup_dict[f'{pept}_{i}']
            modelos.loc[modelos['Peptide']==f'{pept}_{i}', 'Scores'] = scores_dict[f'{pept}_{i}']
    
    df['iteration'] = gen
    modelos = pd.concat([old_model, modelos]).reset_index(drop=True)
    modelos.to_csv('Results/Modelos.csv', index=False)
    df.to_csv(f'Results/contacts_{gen}.csv')
    
    # Gera nova população baseada no peptídeo que teve maior ocupação
    max_ocup_ite = modelos.loc[modelos['Iteration']==gen, 'Ocup'].max()
    pep = modelos.loc[modelos['Ocup']==max_ocup_ite, 'Peptide'].values[0]
    melhor = pep.split('_')[0]

    population = []
    # Precisa definir como será feito no caso de não ser aleatório - A escolha de quantas modificações 
    # e de quais resíduos precisarão ser modificados. E como
    while len(population) < pop_size:
        num_changes = np.random.randint(low=1, high=len(melhor))
        new_pep = random_variation(melhor, num_changes)
        if new_pep not in testados:
            population.append(new_pep)
            testados.append(new_pep)

 Matrix: Warning: no convergence (0.00000010<17136.44726562 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<17095.69726562 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<17072.93164062 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<17072.93164062 after 1000 iterations).
 Matrix: Warning: no convergence (0.00000010<19628.11328125 after 1000 iterations).


# Script Anterior

In [6]:
def Potter(sequencias, iterations = 10, RBM = None):
    # Create directories
    if not os.path.isdir("Results/"):
            os.mkdir("Results/")
    if not os.path.isdir("Temp/"):
            os.mkdir("Temp/")
    if not os.path.isdir("Inputs/PDB_docking/"):
            os.mkdir("Results/PDB_docking/")
    if not os.path.isdir("Results/PDB_predicted/"):
            os.mkdir("Results/PDB_predicted/")
    if not os.path.isdir("Results/PDB_complex/"):
            os.mkdir("Results/PDB_complex/")
    
    # Create results files
    summary = open("Results/summary.csv", mode="w+") # Create the summary csv file for later visualization of results
    p_info = open("Results/populations.csv", mode="w+") # Create populations csv file for posterior checks
    summary.write("generation,best_ind,best_gen_fit,avg_gen_fit,worst_gen_fit,total_time" + "\n") # The headers of summary
    p_info.write(p_info.write("generation,sequence,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,best,occupancy"))  # The headers of summary
    p_info.close()
    summary.close()
    
    # Calculate population size as the same size of the initial population. This parameter won't change through generations
    pop_size = len(initial_population)
    population = initial_population
    q = Queue()
    
    # Define the protocol
    protocol = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
    <ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="fa_standard" weights="ref2015.wts"/>
    </SCOREFXNS>
    <MOVERS>
        <FlexPepDock name="ppack"  ppk_only="true"/>
        <FlexPepDock name="fpd" lowres_abinitio="true" pep_refine="true"/>
        <FlexPepDock name="minimize"  min_only="true"/>
    </MOVERS>
    <PROTOCOLS>
        <Add mover="ppack"/>
        <Add mover="fpd"/>
        <Add mover="minimize"/>
    </PROTOCOLS>
    <OUTPUT/>
    </ROSETTASCRIPTS>""").get_mover("ParsedProtocol")
    
    # Define the scorefxn 
    scorefxn = pyrosetta.create_score_function("ref2015") # This is the standard full_atom scoring from pyrosetta
    
    # Initialize lists to keep records of best, worst and average fitness in each generation
    best_fitness = []
    best_sequences = []
    worst_fitness = []
    average_fitness = []
    fit_dict = {}
    upsize = []
    # Calculate fitness for each sequence and fill the records
    for j in range(iterations):
        upsize.append(len(pd.unique(population)))
        # Prepare the directories
        # Tem que mudar para refletir o novo esquema de pastas (Input, Temp e Results)
        if not os.path.isdir("Results/PDB_docking/G" + str(j)):
            os.mkdir("Results/PDB_docking/G" + str(j))
        if not os.path.isdir("Results/PDB_predicted/G" + str(j)):
            os.mkdir("Results/PDB_predicted/G" + str(j))
        if not os.path.isdir("Results/PDB_complex/G" + str(j)):
            os.mkdir("Results/PDB_complex/G" + str(j))
            
        #Start timer to check generation duration
        start = time.time() 
        
        # Preliminary test to check if there are repeated sequences from previous generations
        for sequence in reversed(population): 
            if sequence in fit_dict.keys():
                # Copy the old pdbs to the new folders
                for r in reversed(range(j)):
                    if os.path.isfile("Results/PDB_docking/G" + str(r) + "/dock(" + sequence + ").pdb"):
                        src1 = "Results/PDB_docking/G" + str(r) + "/dock(" + sequence + ").pdb"
                        dst1 = "Results/PDB_docking/G" + str(j) + "/dock(" + sequence + ").pdb"
                        src2 = "Results/PDB_predicted/G" + str(r) + "/" + sequence + ".pdb"
                        dst2 = "Results/PDB_predicted/G" + str(j) + "/" + sequence + ".pdb"
                        src3 = "Results/PDB_complex/G" + str(r) + "/comp(" + sequence + ").pdb"
                        dst3 = "Results/PDB_complex/G" + str(j) + "/comp(" + sequence + ").pdb"
                        shutil.copyfile(src1, dst1)
                        shutil.copyfile(src2, dst2)
                        shutil.copyfile(src3, dst3)
                        break
                population.remove(sequence)
        
        # Organize new population in batches
        batches = makeBatch(population, cores= 1) 
        # Preparation step is sequential because of Modeller
        for batch in batches:
            for sequence in batch:
                if sequence != "BJOUXZ": # The fake peptide speeds up simulation since the first process starts alone
                    if not os.path.isdir("Results/PDB_docking/G0/" + sequence):
                        os.mkdir("Results/PDB_docking/G0/" + sequence)
                    makePeptide(candidate=sequence, gen=j)
            
            # Start the process list
            processes = list()
            for sequence in batch: # The docking step is parallel
                #p = Process(target=fakeFitness_Parallel, args=(sequence, protocol, scorefxn, j, 10, q))
                p = Process(target=calculateFitness_Parallel, args=(sequence, protocol, scorefxn, j, 10, q))
                p.start()
                processes.append(p)

            for p in processes: # We can only move forward after all the processes finish
                p.join()

            for i in range(q.qsize()): # Fetching the scores of each sequence
                [pep, s] = q.get()
                oc = countContacts("Results/PDB_docking/G" + str(j) + "/dock(" + pep + ").pdb", target=RBM)
                if oc < 0.3:
                    s = [9999999999, 9999999999] # Make it so the peptide with less than 30% occupancy gets bad scores 
                fit_dict[pep] = min(s) # The best of 20 scores is selected as true score
            
        fit_list = list()
        for sequence in population:
            fit_list.append(fit_dict[sequence])
            
        best_fitness.append(min(fit_list))
        l = fit_list.index(min(fit_list))
        best_sequences.append(population[l])
        worst_fitness.append(max(fit_list))
        average = sum(fit_list)/len(fit_list)
        average_fitness.append(average)
        end = time.time()
        gen_time = end - start
        summary = open("Results/summary_GA.csv", mode="a") # Open the summary file in append mode
        summary.write(str(j) + "," + str(best_sequences[-1]) + "," + str(best_fitness[-1]) + "," + str(average) + "," + str(worst_fitness[-1]) + "," + str(gen_time) + "\n")
        summary.close()
    return population, best_fitness, worst_fitness, average_fitness, best_sequences, upsize

## Critérios para selecionar os peptídeos

In [ ]:
# Criamos dicionários para qualquer as informações que nos importam do VTR
df = {'Protein':[],
      'Peptide':[],
      'Distance':[],
      'Contact':[]
     }

modelos = {'Peptide':[],
           'Hydrophobic':[],
           'Aromatic Stacking':[],
           'Attractive':[],
           'Hydrogen Bond':[],
           'Repulsive':[],
           'Salt Bridge':[],
}

# Criamos um dicionário para padronizar os tipos de ligação
new = {"HB": 'Hydrogen Bond',
       "HY": 'Hydrophobic',
       "AR": 'Aromatic Stacking',
       "AT": 'Attractive',
       "RE": 'Repulsive',
       "SB": 'Salt Bridge',
      }
d
# Indicamos o nome da pasta onde os resultados do VTR a serem tratados estão
pasta = 'VTR/'

# O Loop/laço for a seguir lê cada linha dos arquivos do VTR e as interpreta
for file in os.listdir(pasta):
    filename = pasta + file
    dist = False
    pep_name = file.split('.')[0].split('_')[1]
    modelos['Peptide'].append(pep_name) #Adicionamos o nome do peptídeo
    # Inicializamos as contagens dos tipos de interação que o VTR identificou
    tipos = {'Hydrophobic':0,
             'Aromatic Stacking':0,
             'Attractive':0,
             'Hydrogen Bond':0,
             'Repulsive':0,
             'Salt Bridge':0,
            }

    # Neste laço/loop for, nós preenchemos nossa tabela com as informações interpretadas
    for line in read_file(filename):
        l = re.split(';|:|-|\n',line) # Dividimos a linha em palavras, ignorando os espaços
        if len(l) == 11 and l[1] in ['A','B']: # O VTR tem o arquivo padronizado. Linhas com informações que queremos tem 11 campos
            if l[1] != l[5]: # Se a cadeia 1 l[2] é diferente da cadeia 2 l[7], processamos
                if l[1] == 'A':
                    df['Protein'].append(' '.join(l[2:4]))
                    df['Peptide'].append(' '.join(l[6:8]))
                else:
                    df['Protein'].append(' '.join(l[6:8]))
                    df['Peptide'].append(' '.join(l[2:4]))
                df['Distance'].append(l[9])
                df['Contact'].append(new[l[0]])
                tipos[new[l[0]]] += 1  # Contabilizamos o tipo de contato encontrado


    # Neste laço/loop for, nós registramos as contagens de tipos de ligação para o modelo
    for k in tipos.keys():
        modelos[k].append(tipos[k])

# CRIAR A TABELA
df = pd.DataFrame(df)
modelos = pd.DataFrame(modelos)

# SALVA OS RESULTADOS
df.to_csv('Tabela_full.csv')
modelos.to_csv('Modelos_full.csv')

## Contabilização dos contatos no sítio de ligação

In [ ]:
#Selecionando quantos residuos da proteina fazem contato unico com o peptídeo. Ocupacao: do toal de residuos que temos na regiao de interesse (sitio de ligacao), quantos desses o peptidio faz contato. 
#entrada são os arquivos do docking hpep.doc
# MARCAMOS OS RESÍDUOS DA RBM QUE ESTÃO NUM ARTIGO PADRONIZADOS
Site = eval(open('interface.txt').readlines()[0])
Site = [f'{x}_{aminoletters[y]}' for x,y in Site]

# PADRONIZAÇÃO DA NOMENCLATURA DO DATAFRAME E CONTAGEM DOS CONTATOS POR RESÍDUO 
prots = []
peps = []
dist = []
tipos = []
aa_prots =[]
aa_peps =[]

for i in range(len(df)):
    a1 = df.iloc[i]['Protein'].split()[0] + '_' + aminoletters[df.iloc[i]['Protein'].split()[1]] 
    aa_a1 = aminoletters[df.iloc[i]['Protein'].split()[1]]
    a2 = df.iloc[i]['Peptide'].split()[0] + '_' + aminoletters[df.iloc[i]['Peptide'].split()[1]]
    aa_a2 = aminoletters[df.iloc[i]['Peptide'].split()[1]]
    prots.append(a1)
    peps.append(a2)
    aa_prots.append(aa_a1)
    aa_peps.append(aa_a2)
    dist.append(float(df.iloc[i]['Distance']))
    tipos.append(df.iloc[i]['Contact'].split(',')[0])

# GERAR TABELA E SALVAR DADOS COMO ARQUIVO CSV 
dados_geral = pd.DataFrame({'Protein':prots,'aa_prot':aa_prots, 'Peptide':peps,'aa_pep':aa_peps, 'Distance':dist, 'Contact':tipos})
dados_site = dados_geral.loc[dados_geral['Protein'].isin(Site)]
dados_site.to_csv('site_contacts_valid.csv') 

In [ ]:
# Criamos dicionários para qualquer as informações que nos importam do VTR
df = {'Proteina':[],
      'Peptideo':[],
      'Distancia':[],
      'Contato':[]
     }

modelos = {'Modelo':[],
           'Hidrofobico':[],
           'Empilhamento Aromatico':[],
           'Atrativa':[],
           'Ligacao de Hidrogenio':[],
           'Repulsiva':[]
}

# Criamos um dicionário para padronizar os tipos de ligação
new = {'Attractive, Hydrogen Bonds': 'Atrativa',
       'Attractive Hydrogen Bonds': 'Atrativa',
       'Repulsivo': 'Repulsiva',
       'Atraente, ligações de hidrogênio':'Atrativa', # Alguns desses nomes estavam nos primeiros arquivos
       'Atraente': 'Atrativa',
       'Empilhamento Aromático': 'Empilhamento Aromatico',
       'Ligações de hidrogênio': 'Ligacao de Hidrogenio',
       'atraente': 'Atrativa',
       'Repulsive': 'Repulsiva',
       'Attractive': 'Atrativa',
       'Aromatic Stacking': 'Empilhamento Aromatico',
       'Hydrogen Bonds': 'Ligacao de Hidrogenio',
       'Hydrophobic': 'Hidrofobico',
       'Repulsive, Hydrogen Bonds':'Repulsiva',
      }


# Indicamos o nome da pasta onde os resultados do VTR a serem tratados estão
pasta = '/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/TODOS_MODELOS/'

# O Loop/laço for a seguir lê cada linha dos arquivos do VTR e as interpreta
for file in os.listdir(pasta):
    filename = pasta + file
    dist = False
    modelos['Modelo'].append(file.split('.')[0]) #Adicionamos o nome do modelo
    # Inicializamos as contagens dos tipos de interação que o VTR identificou
    tipos = {'Hidrofobico':0,
             'Empilhamento Aromatico':0,
             'Atrativa':0,
             'Ligacao de Hidrogenio':0,
             'Repulsiva':0
            }

    # Neste laço/loop for, nós preenchemos nossa tabela com as informações interpretadas
    for line in read_file(filename):
        l = line.split() # Dividimos a linha em palavras, ignorando os espaços
        if dist == True: # Se estamos numa linha referente à interface, lemos a distância e o tipo de contato
            df['Distancia'].append(l[1])
            df['Contato'].append(new[' '.join(l[3:])[:-1]])
            tipos[new[' '.join(l[3:])[:-1]]] += 1  # Contabilizamos o tipo de contato encontrado
            dist = False
        if len(l) == 11: # O VTR tem o arquivo padronizado. Linhas com informações que queremos tem 11 campos
            if l[2] != l[7]: # Se a cadeia 1 l[2] é diferente da cadeia 2 l[7], processamos
                df['Proteina'].append(' '.join(l[3:6]))
                df['Peptideo'].append(' '.join(l[8:11]))
                dist = True # Só então ativamos a leitura da distância, se a condição acima é verdadeira
    
    # Neste laço/loop for, nós registramos as contagens de tipos de ligação para o modelo
    for k in tipos.keys():
        modelos[k].append(tipos[k])

# CRIAR A TABELA
df = pd.DataFrame(df)
modelos = pd.DataFrame(modelos)

# SALVA OS RESULTADOS
df.to_csv('Tabela.csv')
modelos.to_csv('Modelos.csv')

In [ ]:
# MOSTRA CONTAGEM DE TIPOS DE CONTATOS ENCONTRADOS EM TODOS OS ARQUIVOS ANALISADOS
plt.figure(figsize=(12,9))
sns.histplot(y='Contato', data=df, color = 'tab:red', edgecolor='tab:orange')

In [ ]:
# CONTABILIZAÇÃO DOS CONTATOS DA RBD

# DICIONÁRIO PARA CONVERTER CÓDIGO DE 3 LETRAS PARA CÓDIGO DE 1 LETRA
aminoletters = {'ALA':'A',
                'CYS':'C',
                'ASP':'D', 
                'GLU':'E', 
                'PHE':'F', 
                'GLY':'G',
                'HIS':'H',
                'ILE':'I',
                'LYS':'K',
                'LEU':'L',
                'MET':'M',
                'ASN':'N',
                'PRO':'P',
                'GLN':'Q',
                'ARG':'R',
                'SER':'S',
                'THR':'T',
                'VAL':'V',
                'TRP':'W',
                'TYR':'Y',
                'LN':'Q' # Isso foi um erro que tinha em algum arquivo passado
                }

# PADRONIZAÇÃO DA NOMENCLATURA DO DATAFRAME E CONTAGEM DOS CONTATOS POR RESÍDUO 
prots = []
peps = []
dist = []
tipos = []
aa_prots =[]
aa_peps =[]

for i in range(len(df)):
    a1 = df.iloc[i]['Proteina'].split()[1] + '_' + aminoletters[df.iloc[i]['Proteina'].split()[0]] 
    aa_a1 = aminoletters[df.iloc[i]['Proteina'].split()[0]]
    a2 = df.iloc[i]['Peptideo'].split()[1] + '_' + aminoletters[df.iloc[i]['Peptideo'].split()[0]]
    aa_a2 = aminoletters[df.iloc[i]['Peptideo'].split()[0]]
    prots.append(a1)
    peps.append(a2)
    aa_prots.append(aa_a1)
    aa_peps.append(aa_a2)
    dist.append(float(df.iloc[i]['Distancia']))
    tipos.append(df.iloc[i]['Contato'].split(',')[0])

# GERAR TABELA E SALVAR DADOS COMO ARQUIVO CSV 
dados = pd.DataFrame({'Proteina':prots,'aa_prot':aa_prots, 'Peptideo':peps,'aa_pep':aa_peps, 'Distancia':dist, 'Contato':tipos})
#dados.to_csv('/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/dados_rbd_total.csv')

# PLOTAGEM DO HISTOGRAMA DE CONTATOS
plt.figure(figsize=(30,30))
sns.histplot(x = prots, y=sorted(peps), hue=tipos, palette=['#FA160E','#511AD6','#6AD61A','#FAB41E','#28EAED'])
#sns.jointplot(x = prots, y=sorted(peps), kind='hex', hue=tipos, palette=['#FA160E','#511AD6','#6AD61A','#FAB41E','#28EAED'])
plt.ylabel('Peptídeo')
plt.xlabel('Proteina')
plt.grid()
plt.show()

In [ ]:
# CONTABILIZAÇÃO DOS CONTATOS DA RBM
#Selecionando quantos residuos da proteina fazem contato unico com o peptídio. Ocupacao: do toal de residuos que temos na regiao de interesse (sitio de ligacao), quantos desses o peptidio faz contato. 
#entrada são os arquivos do docking hpep.doc
# MARCAMOS OS RESÍDUOS DA RBM QUE ESTÃO NUM ARTIGO PADRONIZADOS
RBM = []
for line in read_file ('/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/VTRNOVOS/RBM.txt'):
    l = eval(line)
    RBM.append("{}_{}".format(l[0],aminoletters[l[1]]))

# PADRONIZAÇÃO DA NOMENCLATURA DO DATAFRAME E CONTAGEM DOS CONTATOS POR RESÍDUO 
prots = []
peps = []
dist = []
tipos = []
aa_prots =[]
aa_peps =[]

for i in range(len(df)):
    a1 = df.iloc[i]['Proteina'].split()[1] + '_' + aminoletters[df.iloc[i]['Proteina'].split()[0]] 
    aa_a1 = aminoletters[df.iloc[i]['Proteina'].split()[0]]
    a2 = df.iloc[i]['Peptideo'].split()[1] + '_' + aminoletters[df.iloc[i]['Peptideo'].split()[0]]
    aa_a2 = aminoletters[df.iloc[i]['Peptideo'].split()[0]]
    prots.append(a1)
    peps.append(a2)
    aa_prots.append(aa_a1)
    aa_peps.append(aa_a2)
    dist.append(float(df.iloc[i]['Distancia']))
    tipos.append(df.iloc[i]['Contato'].split(',')[0])
    #ranking ordenado por contato (ocupacao). desempate: score(menor score->libera mais energia (positivo ou neg). Quanto mais neg melhor, mais forte a ligacao. Menor score, melhor)
    #seleciona os n(2) melhores. Passa esses dois para o VTR(codigo dos meninos do lab). 
    # vtr pega os contatos e mede a distancia. (como?) Gera uma visualizacao com os contatos mais frequentes
    #

# GERAR TABELA E SALVAR DADOS COMO ARQUIVO CSV 
dados_geral = pd.DataFrame({'Proteina':prots,'aa_prot':aa_prots, 'Peptideo':peps,'aa_pep':aa_peps, 'Distancia':dist, 'Contato':tipos})
dados_rbm = dados_geral.loc[dados_geral['Proteina'].isin(RBM)]
dados_rbm.to_csv('/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/dados_rbm_total.csv') 

# PLOTAGEM DO HISTOGRAMA DE CONTATOS
plt.figure(figsize=(30,30))
sns.histplot(data=dados_rbm, x = 'Proteina', y='Peptideo', hue='Contato', palette=['#FA160E','#511AD6','#28EAED','#6AD61A','#FAB41E'])
plt.ylabel('Peptídeo')
plt.xlabel('Proteina')
plt.grid()
plt.show()

In [ ]:
# MAPA DE CALOR DA FREQUÊNCIA DE CONTATOS AMINOÁCIO-AMINOÁCIDO
cols = {'A':[0]*20,'C':[0]*20,'D':[0]*20,'E':[0]*20,'F':[0]*20,'G':[0]*20,'H':[0]*20,'I':[0]*20,
        'K':[0]*20,'L':[0]*20,'M':[0]*20,'N':[0]*20,'P':[0]*20,'Q':[0]*20,'R':[0]*20,'S':[0]*20,
        'T':[0]*20,'V':[0]*20,'W':[0]*20,'Y':[0]*20}

index = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
matrix = pd.DataFrame(cols, index= index)
for i in pd.unique(list(aminoletters.values())):
    for j in pd.unique(list(aminoletters.values())):
        n = len((dados.loc[(dados['aa_prot']==i) & (dados['aa_pep']==j) | (dados['aa_prot']==j) & (dados['aa_pep']==i),'Contato']))
        matrix[i][j] = n
        matrix[j][i] = n

figure, ax = plt.subplots(figsize= (16,14))
ax.xaxis.set_ticks_position('bottom')
ax.xaxis.set_label_position('bottom')
mask = np.zeros_like(matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True
np.fill_diagonal(mask, False)
sns.heatmap(matrix, ax = ax, cmap = 'RdBu_r', annot=True, fmt='g', mask=mask)
plt.show()

In [ ]:
# VERIFICAÇÃO DA FREQUÊNCIA DE CONTATOS AMINOÁCIO-RBD (TODOS OS PEPTÍDEOS)
index = sorted(pd.unique(dados['Proteina']))
x = len(index)
cols = {'A':[0]*x,'C':[0]*x,'D':[0]*x,'E':[0]*x,'F':[0]*x,'G':[0]*x,'H':[0]*x,'I':[0]*x,
        'K':[0]*x,'L':[0]*x,'M':[0]*x,'N':[0]*x,'P':[0]*x,'Q':[0]*x,'R':[0]*x,'S':[0]*x,
        'T':[0]*x,'V':[0]*x,'W':[0]*x,'Y':[0]*x}

matrix = pd.DataFrame(cols, index=index)
for i in pd.unique(list(cols.keys())):
    for j in index:
        n = len(dados.loc[(dados['Proteina']==j) & (dados['aa_pep']==i)])
        matrix.loc[j][i] = n
        #matrix[i].loc[j] = n

matrix = matrix.transpose()
figure, ax = plt.subplots(figsize= (16,14))
ax.xaxis.set_ticks_position('bottom')
ax.xaxis.set_label_position('bottom')
sns.heatmap(matrix, ax = ax, cmap ='RdBu_r', annot=True, fmt='g')
plt.show()
# SALVAR MATRIZ GERADA
#matrix.to_csv('/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/matrix_total.csv')

In [ ]:
# Criamos dicionários para qualquer as informações que nos importam do VTR
df_literatura = {'Proteina':[],
      'Peptideo':[],
      'Distancia':[],
      'Contato':[]
     }

modelos_literatura = {'Modelo':[],
           'Hidrofobico':[],
           'Empilhamento Aromatico':[],
           'Atrativa':[],
           'Ligacao de Hidrogenio':[],
           'Repulsiva':[]
}

# Criamos um dicionário para padronizar os tipos de ligação
new = {'Attractive, Hydrogen Bonds': 'Atrativa',
       'Attractive Hydrogen Bonds': 'Atrativa',
       'Repulsivo': 'Repulsiva',
       'Atraente, ligações de hidrogênio':'Atrativa', # Alguns desses nomes estavam nos primeiros arquivos
       'Atraente': 'Atrativa',
       'Empilhamento Aromático': 'Empilhamento Aromatico',
       'Ligações de hidrogênio': 'Ligacao de Hidrogenio',
       'atraente': 'Atrativa',
       'Repulsive': 'Repulsiva',
       'Attractive': 'Atrativa',
       'Aromatic Stacking': 'Empilhamento Aromatico',
       'Hydrogen Bonds': 'Ligacao de Hidrogenio',
       'Hydrophobic': 'Hidrofobico',
       'Repulsive, Hydrogen Bonds':'Repulsiva',
      }


# Indicamos o nome da pasta onde os resultados do VTR a serem tratados estão
pasta = '/content/drive/MyDrive/MATERIAL COMPLETO ANA PAULA ABREU VTR E HPEPDOCK/TODOS_LITERATURA/'

# O Loop/laço for a seguir lê cada linha dos arquivos do VTR e as interpreta
for file in os.listdir(pasta):
    filename = pasta + file
    dist = False
    modelos_literatura['Modelo'].append(file.split('.')[0]) #Adicionamos o nome do modelo
    # Inicializamos as contagens dos tipos de interação que o VTR identificou
    tipos = {'Hidrofobico':0,
             'Empilhamento Aromatico':0,
             'Atrativa':0,
             'Ligacao de Hidrogenio':0,
             'Repulsiva':0
            }

    # Neste laço/loop for, nós preenchemos nossa tabela com as informações interpretadas
    for line in read_file(filename):
        l = line.split() # Dividimos a linha em palavras, ignorando os espaços
        if dist == True: # Se estamos numa linha referente à interface, lemos a distância e o tipo de contato
            df_literatura['Distancia'].append(l[1])
            df_literatura['Contato'].append(new[' '.join(l[3:])[:-1]])
            tipos[new[' '.join(l[3:])[:-1]]] += 1  # Contabilizamos o tipo de contato encontrado
            dist = False
        if len(l) == 11: # O VTR tem o arquivo padronizado. Linhas com informações que queremos tem 11 campos
            if l[2] != l[7]: # Se a cadeia 1 l[2] é diferente da cadeia 2 l[7], processamos
                df_literatura['Proteina'].append(' '.join(l[3:6]))
                df_literatura['Peptideo'].append(' '.join(l[8:11]))
                dist = True # Só então ativamos a leitura da distância, se a condição acima é verdadeira
    
    # Neste laço/loop for, nós registramos as contagens de tipos de ligação para o modelo
    for k in tipos.keys():
        modelos_literatura[k].append(tipos[k])

# CRIAR A TABELA
df_literatura = pd.DataFrame(df_literatura)
modelos_literatura = pd.DataFrame(modelos_literatura)
modelos_literatura.sort_values('Modelo', axis=0, inplace=True)
# SALVA OS RESULTADOS
df_literatura.to_csv('Tabela_literatura.csv')
modelos_literatura.to_csv('Modelos_literatura.csv')